In [1]:
import dask.dataframe as ddf
import datetime
import G211
import logging
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pdb
import ml_functions
from scipy import spatial
import time
import xarray

/glade/u/home/ahijevyc/.local/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/glade/u/home/ahijevyc/.local/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
2024-09-20 09:49:38.137195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 09:49:38.735691: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on.

In [2]:
parser = ml_functions.get_argparser()
args, argv = parser.parse_known_args()
args.batchnorm = True
args.model = "HRRR"
args.epochs = 10
args.debug = False
args.kfold = 1
args.learning_rate = 0.01
args.neurons = [32, 32]
args.optimizer = "SGD"
args.suite = "with_storm_mode"
args.teststart = pd.to_datetime("19000101")
args.testend = pd.to_datetime("20220101")
twin = 4
# Ryan's original way of naming storm rpt time windows
rpttwin = int(np.floor(twin / 2))
args.labels = [
    f"sighail_40km_{rpttwin}hr",
    f"sigwind_40km_{rpttwin}hr",
    f"hailone_40km_{rpttwin}hr",
    f"wind_40km_{rpttwin}hr",
    f"torn_40km_{rpttwin}hr",
    f"any_40km_{rpttwin}hr",
]
engine = ddf

## Load or create DataFrame of features and labels

In [4]:
new = ml_functions.load_df(
    args,
    idir = "/work/sobash/NSC_objects",
    wbugdir = "wbug_lightning"
)

FileNotFoundError: [Errno 2] No such file or directory: '/glade/work/ahijevyc/NSC_objects/predictor_suites/HRRR.with_storm_mode.txt'

In [17]:
#can't read with dask because it divides total memory by number of workers
old = pd.read_parquet(Path(os.getenv("TMPDIR"))/f"HRRR/HRRR.rptdist_{rptdist}km.par")
old.shape

(45777384, 391)

In [17]:
new = engine.read_parquet(Path(os.getenv("TMPDIR"))/"u.par")

In [18]:
set(new.columns) - set(old.columns)

{'valid_time', 'x', 'y'}

In [19]:
# old parquet has storm mode probabilities. do I need those?
set(old.columns) - set(new.columns)

{'CNN_1_Disorganized',
 'CNN_1_Disorganized_nprob',
 'CNN_1_Disorganized_prob',
 'CNN_1_QLCS',
 'CNN_1_QLCS_nprob',
 'CNN_1_QLCS_prob',
 'CNN_1_Supercell',
 'CNN_1_Supercell_nprob',
 'CNN_1_Supercell_prob',
 'DNN_1_Disorganized',
 'DNN_1_Disorganized_nprob',
 'DNN_1_Disorganized_prob',
 'DNN_1_QLCS',
 'DNN_1_QLCS_nprob',
 'DNN_1_QLCS_prob',
 'DNN_1_Supercell',
 'DNN_1_Supercell_nprob',
 'DNN_1_Supercell_prob',
 'SS_Disorganized',
 'SS_Disorganized_nprob',
 'SS_Disorganized_prob',
 'SS_QLCS',
 'SS_QLCS_nprob',
 'SS_QLCS_prob',
 'SS_Supercell',
 'SS_Supercell_nprob',
 'SS_Supercell_prob',
 'initialization_time'}

In [20]:
new = new.compute()

KeyError: "['flashes_40km_4hr', 'flashes_40km_1hr', 'flashes_40km_2hr'] not in index"

In [ ]:
# you can't set_index(["y","x"], append=True) unless you compute first. (dask doesn't deal with MultiIndex)
new = new.set_index(["y","x"], append=True).sort_index()

In [ ]:
old.shape,new.shape

In [ ]:
old.index.is_monotonic_increasing

In [ ]:
new = new.sort_index()

In [ ]:
new.index.is_monotonic_increasing

In [ ]:
old.index.equals(new.index)

In [ ]:
old.initialization_time.equals(new.initialization_time)

In [ ]:
old.initialization_time

In [ ]:
new.initialization_time

In [ ]:
(old.initialization_time != new.initialization_time).idxmax()

In [ ]:
old.loc[('2019-10-04 01:00:00', 12, 49)]

In [ ]:
new.loc[('2019-10-04 01:00:00', 12, 49)]